In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sarcoma"
cohort = "GSE159848"

# Input paths
in_trait_dir = "../../input/GEO/Sarcoma"
in_cohort_dir = "../../input/GEO/Sarcoma/GSE159848"

# Output paths
out_data_file = "../../output/preprocess/Sarcoma/GSE159848.csv"
out_gene_data_file = "../../output/preprocess/Sarcoma/gene_data/GSE159848.csv"
out_clinical_data_file = "../../output/preprocess/Sarcoma/clinical_data/GSE159848.csv"
json_path = "../../output/preprocess/Sarcoma/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the series description and overall design, this dataset contains gene expression data from microarray
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (sarcoma):
# Looking at the sample characteristics, all samples are mixoid liposarcoma patients (row 2)
# Since all samples are sarcoma patients, we need a binary trait for case-control analysis
# We'll use the metastasis status (row 3) as our trait of interest since it has binary values (0, 1)
trait_row = 3

# For age:
# Age is available in row 1
age_row = 1

# For gender:
# Gender (Sex) is available in row 0
gender_row = 0

# 2.2 Data Type Conversion

def convert_trait(value: str) -> Optional[int]:
    """Convert metastasis status to binary."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return int(value)  # 0 for no metastasis, 1 for metastasis
    except (ValueError, TypeError):
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age to continuous numeric value."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to lowercase for case-insensitive comparison
    value = value.lower()
    
    if value == 'f' or value == 'female':
        return 0
    elif value == 'm' or value == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # In previous step the clinical data was parsed and is available in memory
    # We need to get it from the sample characteristics dictionary
    # Convert the dictionary to a DataFrame
    clinical_dict = {0: ['Sex: M', 'Sex: F'], 
                    1: ['age: 44', 'age: 67', 'age: 54', 'age: 82', 'age: 47', 'age: 32', 'age: 57', 
                        'age: 60', 'age: 51', 'age: 45', 'age: 38', 'age: 16', 'age: 52', 'age: 46', 
                        'age: 58', 'age: 20', 'age: 39', 'age: 43', 'age: 31', 'age: 71', 'age: 49', 
                        'age: 28', 'age: 29', 'age: 75', 'age: 74', 'age: 40', 'age: 59', 'age: 42', 
                        'age: 35', 'age: 33'], 
                    2: ['subject status/id: mixoid liposarcoma patient 1', 'subject status/id: mixoid liposarcoma patient 2', 
                        'subject status/id: mixoid liposarcoma patient 3', 'subject status/id: mixoid liposarcoma patient 4', 
                        'subject status/id: mixoid liposarcoma patient 5', 'subject status/id: mixoid liposarcoma patient 6', 
                        'subject status/id: mixoid liposarcoma patient 7', 'subject status/id: mixoid liposarcoma patient 8', 
                        'subject status/id: mixoid liposarcoma patient 9', 'subject status/id: mixoid liposarcoma patient 10', 
                        'subject status/id: mixoid liposarcoma patient 11', 'subject status/id: mixoid liposarcoma patient 12', 
                        'subject status/id: mixoid liposarcoma patient 13', 'subject status/id: mixoid liposarcoma patient 14', 
                        'subject status/id: mixoid liposarcoma patient 15', 'subject status/id: mixoid liposarcoma patient 16', 
                        'subject status/id: mixoid liposarcoma patient 17', 'subject status/id: mixoid liposarcoma patient 18', 
                        'subject status/id: mixoid liposarcoma patient 19', 'subject status/id: mixoid liposarcoma patient 20', 
                        'subject status/id: mixoid liposarcoma patient 21', 'subject status/id: mixoid liposarcoma patient 22', 
                        'subject status/id: mixoid liposarcoma patient 23', 'subject status/id: mixoid liposarcoma patient 24', 
                        'subject status/id: mixoid liposarcoma patient 25', 'subject status/id: mixoid liposarcoma patient 26', 
                        'subject status/id: mixoid liposarcoma patient 27', 'subject status/id: mixoid liposarcoma patient 28', 
                        'subject status/id: mixoid liposarcoma patient 29', 'subject status/id: mixoid liposarcoma patient 30'], 
                    3: ['metastasis: 0', 'metastasis: 1'], 
                    4: ['time_mfs: 9.78630136986301', 'time_mfs: 0.463013698630137', 'time_mfs: 13.9150684931507', 
                        'time_mfs: 7.37534246575343', 'time_mfs: 11.9095890410959', 'time_mfs: 8.65753424657534', 
                        'time_mfs: 8.10684931506849', 'time_mfs: 6.52054794520548', 'time_mfs: 7.06301369863014', 
                        'time_mfs: 4.65753424657534', 'time_mfs: 6.75068493150685', 'time_mfs: 1.29315068493151', 
                        'time_mfs: 4.33150684931507', 'time_mfs: 4.50958904109589', 'time_mfs: 2.83835616438356', 
                        'time_mfs: 3.41917808219178', 'time_mfs: 1.70684931506849', 'time_mfs: 0.687671232876712', 
                        'time_mfs: 0.520547945205479', 'time_mfs: 0.501369863013699', 'time_mfs: 0.476712328767123', 
                        'time_mfs: 0.591780821917808', 'time_mfs: 0.441095890410959', 'time_mfs: 0.180821917808219', 
                        'time_mfs: 0.0575342465753425', 'time_mfs: 0.747945205479452', 'time_mfs: 0.380821917808219


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:

import os
import pandas as pd
import json
import numpy as np
import gzip
from typing import Dict, Any, Optional, Callable

# Function to read gzipped matrix file
def read_gzipped_matrix(file_path):
    with gzip.open(file_path, 'rt') as f:
        lines = f.readlines()
    return lines

# Load the matrix file (handling gzipped version)
matrix_file = os.path.join(in_cohort_dir, f"{cohort}_series_matrix.txt.gz")
if os.path.exists(matrix_file):
    # Read the gzipped file
    lines = read_gzipped_matrix(matrix_file)
    
    # Extract metadata section and data section
    metadata_lines = []
    data_start_idx = None
    for i, line in enumerate(lines):
        if line.startswith('!series_matrix_table_begin'):
            data_start_idx = i + 1
            break
        metadata_lines.append(line)
    
    # Extract sample characteristics
    sample_char_lines = [line for line in metadata_lines if line.startswith('!Sample_characteristics_ch1')]
    
    if sample_char_lines:
        # Parse sample characteristics into a dataframe
        sample_chars = []
        for line in sample_char_lines:
            parts = line.strip().split('\t')
            sample_chars.append(parts[1:])
        
        clinical_data = pd.DataFrame(sample_chars)
        
        # Print unique values for each row to identify trait, age, and gender
        print("Examining sample characteristics rows:")
        for i in range(len(clinical_data.index)):
            unique_values = clinical_data.iloc[i].unique()
            print(f"Row {i} unique values: {unique_values}")
    
    # Check if there's a gene expression data section
    if data_start_idx is not None:
        data_line = lines[data_start_idx].strip()
        data_cols = data_line.split('\t')
        first_data_line = lines[data_start_idx + 1].strip().split('\t')
        
        print("\nFirst few data columns:")
        for i in range(min(5, len(data_cols))):
            print(f"{data_cols[i]}: {first_data_line[i] if i < len(first_data_line) else 'N/A'}")
else:
    print(f"Matrix file {matrix_file} not found.")
    clinical_data = pd.DataFrame()

# 1. Gene Expression Data Availability
# Check if there's data that looks like gene expression
is_gene_available = True if data_start_idx is not None else False

# 2.1 Data Availability
# Based on the examination of the sample characteristics:
# Row 0: Sex information
# Row 1: Age information
# Row 3: Metastasis status (binary, can be used as the trait)
gender_row = 0  # Sex information
age_row = 1     # Age information
trait_row = 3   # Using metastasis status as the trait variable

# Validate that these rows exist and contain useful data
if len(clinical_data.index) <= trait_row or len(clinical_data.iloc[trait_row].unique()) <= 1:
    trait_row = None
if len(clinical_data.index) <= age_row or len(clinical_data.iloc[age_row].unique()) <= 1:
    age_row = None
if len(clinical_data.index) <= gender_row or len(clinical_data.iloc[gender_row].unique()) <= 1:
    gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert metastasis status to binary
    value = value.strip('"')  # Remove quotes
    try:
        return int(value)  # Already 0 or 1
    except:
        return None

def convert_age(value):
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Remove quotes
    value = value.strip('"')
    
    # Try to extract numeric age
    import re
    age_match = re.search(r'\d+', value)
    if age_match:
        return float(age_match.group())
    return None

def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Remove quotes
    value = value.strip('"')
    
    # Convert to binary (0: female, 1: male)
    lower_value = value.lower()
    if 'female' in lower_value or 'f' == lower_value:
        return 0
    elif 'male' in lower_value or 'm' == lower_value:
        return 1
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_data, trait, trait_row, convert_trait,
        age_row, convert_age if age_row is not None else None,
        gender_row, convert_gender if gender_row is not None else None
    )
    
    # Preview the selected clinical features
    print("\nPreview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("No trait data available, skipping clinical feature extraction.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


### Step 5: Gene Identifier Review

In [ ]:
# Based on the gene identifiers shown, these don't appear to be standard human gene symbols
# The identifiers (like A_23_P100001) look like Agilent microarray probe IDs
# These will need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns contain gene identifiers and gene symbols
# Looking at the gene annotation preview and gene expression data:
# - 'ID' in gene_annotation contains probe identifiers (e.g., A_23_P100001)
# - 'GENE_SYMBOL' contains human gene symbols (e.g., FAM174B)

# 2. Extract the relevant columns for mapping
probe_id_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted gene expression data shape: {gene_data.shape}")
print("First few rows of gene expression data:")
print(gene_data.head())

# Normalize gene symbols
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Final gene expression data shape after normalization: {gene_data.shape}")
print("First few rows after normalization:")
print(gene_data.head())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. First, we'll reload the clinical data that was processed in Step 3
# This contains the metastasis status, age, and gender information
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data from Step 3, shape: {clinical_data.shape}")

# If the current clinical data is just the "Sarcoma" variable, let's recreate it
# based on the analysis we did in Step 3
if clinical_data.shape[0] == 1 and "Sarcoma" in clinical_data.index:
    print("Recreating clinical data with metastasis, age, and gender...")
    
    # Get the matrix file to extract the sample characteristics data
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get background info and clinical data from the matrix file
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_raw = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Define the conversion functions from Step 3
    def convert_trait(value):
        if value is None or pd.isna(value):
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        value = value.strip('"')
        try:
            return int(value)  # Already 0 or 1
        except:
            return None

    def convert_age(value):
        if value is None or pd.isna(value):
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        value = value.strip('"')
        import re
        age_match = re.search(r'\d+', value)
        if age_match:
            return float(age_match.group())
        return None

    def convert_gender(value):
        if value is None or pd.isna(value):
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        value = value.strip('"')
        lower_value = value.lower()
        if 'female' in lower_value or 'f' == lower_value:
            return 0
        elif 'male' in lower_value or 'm' == lower_value:
            return 1
        return None
    
    # Extract clinical features using the row indices identified in Step 3
    trait_row = 3  # Metastasis status
    age_row = 1    # Age information
    gender_row = 0  # Sex information
    
    # Use "Metastasis" as the trait name since we're using metastasis status
    trait_name = "Metastasis"
    
    clinical_data = geo_select_clinical_features(
        clinical_raw, trait_name, trait_row, convert_trait,
        age_row, convert_age, gender_row, convert_gender
    )
    
    # Save the updated clinical data
    clinical_data.to_csv(out_clinical_data_file)
    print(f"Updated clinical data saved to {out_clinical_data_file}")
    print("Clinical data sample:")
    print(preview_df(clinical_data))

print(f"Clinical data shape: {clinical_data.shape}")

# 2. Load the gene expression data
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded gene expression data shape: {gene_data.shape}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data, gene_data)
print(f"Shape of linked data: {linked_data.shape}")
print("Linked data sample (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 4. Handle missing values in the linked data
# The trait variable is "Metastasis" (not "Sarcoma")
trait_name = "Metastasis" if "Metastasis" in linked_data.columns else clinical_data.index[0]
linked_data_cleaned = handle_missing_values(linked_data, trait_name)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait_name)

# 6. Validate the dataset and save cohort information
note = "Dataset contains expression data for myxoid liposarcoma patients. Metastasis status (0=no metastasis, 1=metastasis) is used as the trait variable for association studies."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")